In [1]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools, LinearAlgebra, GraphPlot, Arpack, Plots, SparseArrays
using Statistics
using Measures
using StatsBase

In [15]:
rng = Random.default_rng();

# graph parameters
n = 50
p = 0.9
eta = 0.

# planted ranking score
planted_score = randperm(rng,n)

# graph model
type = "MUN" 

if type=="MUN"
    meta_g = gen_graph_mun(rng, n, p, eta;planted_score)
elseif type=="ERO"
    meta_g = gen_graph_ero(rng, n, p, eta;planted_score)
end


{50, 1104} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [16]:
batch = 5

# technical parameters
weighted = true # normalized Laplacian is used
singular = true # all eigenpairs are computed for more stability

# incidence matrix
B = magnetic_incidence(meta_g)

# full magnetic Laplacian
W = I # weight matrix
if weighted
    e_weights = get_edges_prop(meta_g, :e_weight, true, 1.0)
    W *= diagm(e_weights)
end
L = B * W * B'

# normalization of L
normalize_Lap!(L)

# least eigenvector full Laplacian
v = least_eigenvector(L; singular)

# recovered ranking full Laplacian
ranking_full = syncrank(L, meta_g; singular)

planted_ranking = ranking_from_score(planted_score)
tau_full = corkendall(planted_ranking, ranking_full)

0.9755102040816327